# Time dependent parameter generation
In order to perform sensitivity analysis, we need the time dependent state values of matrix `X`. We can find these with the following steps:
1. Load in the state space model
2. Integrate those functions with respect to time
3. Enter the time dependent values of the parameters
4. Solve the system of equations for some time range

**Note**:

If this doesnt work, integrate the state space model from t=0 to t=T then solve the system of equations and see which one works

In [3]:
import sympy as sp

ssm_file = '/Users/zacharymcgrath/Documents/School/Spring 2020/Dynamic Systems in Biology/dynamic_models_in_bio_final_project/state_space_derivation.txt'
eqts = []
xs = []
thetas = []

theta_char = 'k'
extras = ['(', ')', '[', ']']

def get_symbols(string):
    symbols = []
    nopows = string.split('**')
    for nopow in nopows:
        nodivs = nopow.split('/')
        for nodiv in nodivs:
            nomults = nodiv.split('*')
            for nomult in nomults:
                noadds = nomult.split('+')
                for noadd in noadds:
                    nosub = noadd.split('-')
                    for s in nosub:
                        for ext in extras:
                            s = s.replace(ext, '')
                        symbols.append(s)
    return symbols

with open(ssm_file, 'r') as o:
    for line in o:
        x = line.split('=')[0].split('(')[0]
        if x not in xs:
            xs.append(x)
        eqt = line.split('=')[1].strip()
        symbols = get_symbols(eqt)
        for s in symbols:
            if theta_char in s and s not in thetas:
                thetas.append(s)
                
        eqts.append((x, eqt))
    
thetas.sort(key=lambda x: int(x[1:]))   

## Integrate all equations with respect to time

In [4]:
integrated = []
# since all terms don't have t in them, simply multiply by t?
for i in range(len(xs)):
    yt = sp.Mul(sp.sympify(eqts[i][1]), sp.sympify('t'))
    integrated.append((xs[i] + '(t)', yt))
print(integrated[0])


('x1(t)', t*(-k1*x1*x2 + k2*x3 - k34*x1*x10 + k35*x8 + k36*x22 - k37*x1 - k38*x1 + k39*x16))


### Load in the values of all of the coefficients and evaluate

In [5]:
cs = {}
c_file = '/Users/zacharymcgrath/Documents/School/Spring 2020/Dynamic Systems in Biology/dynamic_models_in_bio_final_project/reaction_coefficients.txt'

def get_val(string):
    exp = float(string.split('**')[1]) if '**' in string else 1
    p = float(string.split('*')[1]) if '*' in string else 1
    val = float(string.split('*')[0])
    return (p ** exp) * val

with open(c_file, 'r') as o:
    for line in o:
        k = line.split('=')[0]
        val = line.split('=')[1]
        cs[k] = get_val(val)

print(cs['k1'])
print(cs['k20'])

0.5
8e-05


## load the initial conditions and run for T timesteps

In [6]:
#load the initial conditions
init_cond_f = '/Users/zacharymcgrath/Documents/School/Spring 2020/Dynamic Systems in Biology/dynamic_models_in_bio_final_project/initial_conditions.txt'

init_cond = {}
def get_val(string):
    exp = float(string.split('**')[1]) if '**' in string else 1
    p = float(string.split('*')[1]) if '*' in string else 1
    val = float(string.split('*')[0])
    return (p ** exp) * val

with open(init_cond_f, 'r') as o:
    for line in o:
        k = line.split('=')[0]
        val = line.split('=')[1]
        init_cond[k] = get_val(val)


In [20]:
from copy import deepcopy

T = 400

just_eqts = [eqt[1] for eqt in eqts]
xs_t = [[x for x in range(len(xs))] for _ in range(T)]

for t in range(T):
    print('time step {}\r'.format(t), end='')
    constants = [cs[thetas[i]] * (t + 1) for i in range(len(thetas))]
        
    # copy the equations and substitute the new theta in 
    es = []
    for j, eq in enumerate(just_eqts):
        e = deepcopy(eq)
        for i, theta in enumerate(thetas):
            e = sp.sympify(e).subs(theta, constants[i])
        es.append(e)
        
    spxs = [sp.sympify(x) for x in xs]
    print('solving....')
    xs_t[t] = sp.solve(es, spxs)
    print('done')


solving....


KeyboardInterrupt: 